<a href="https://colab.research.google.com/github/AKookani/Minimized-Genetic-Algorithm/blob/main/Regression_With_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
import copy
import math

# Define a Node class for the tree structure
class Node:
    def __init__(self, nodevalue):
        self.nodevalue = nodevalue  # The value (operator or operand) of the node
        self.right = None  # Right child
        self.left = None  # Left child
        self.MSE = None  # Mean Squared Error of the tree rooted at this node

    def __lt__(self, other):
        # Ensure MSE is a float to avoid TypeError when comparing with complex numbers
        return float(self.MSE) < float(other.MSE)  # Allow sorting by MSE

def print_tree(root):
    """ Recursively print the tree as a mathematical expression. """
    if root is None:
        return ""
    if root.left is None and root.right is None:
        return str(root.nodevalue)  # Leaf node (operand)

    left = print_tree(root.left)
    right = print_tree(root.right)

    if root.nodevalue in ['+', '-', '*', '/', '**']:
        return f'({left}{root.nodevalue}{right})'  # Binary operator
    return f'{root.nodevalue}({left})'  # Unary operator

def combine_trees(tree1, tree2):
    """ Combine two trees by replacing the leftmost 'x' node in tree1 with tree2. """
    temp = copy.deepcopy(tree1)
    while temp.left:
        if temp.left.nodevalue == 'x':
            temp.left = tree2
            break
        temp = temp.left
    return temp

def mutation(tree, operators, operands, mutation_prob=0.1):
    """
    Apply mutation to the tree with a given probability.

    This function randomly mutates nodes within the tree, either changing
    operators or operands, to introduce diversity in the population.
    """

    def mutate_node(node):
        """ Mutate a single node based on mutation probability. """
        # Generate a random number; if greater than mutation_prob, do nothing
        if random.random() > mutation_prob:
            return

        # If the node's value is an operator, mutate by selecting a new operator
        if node.nodevalue in operators:
            node.nodevalue = random.choice(operators)
        else:
            # If the node's value is an operand, mutate by selecting a new operand
            node.nodevalue = random.choice(operands)

    # Initialize a stack with the root of the tree for depth-first traversal
    stack = [tree]

    # Traverse the tree iteratively using a stack
    while stack:
        # Pop the last node from the stack (LIFO - depth-first)
        node = stack.pop()

        if node:
            # Attempt to mutate the current node
            mutate_node(node)

            # Add the left and right children to the stack if they exist
            stack.extend(filter(None, [node.left, node.right]))

def calculate_tree(tree, x_vals):
    """
    Evaluate the tree for each x value and return predictions.

    This function generates a mathematical expression from the tree,
    evaluates it for each value in `x_vals`, and returns the predicted results.
    """

    # Convert the tree into a string representation of the mathematical expression
    func_str = print_tree(tree)

    # Initialize an empty list to store the predictions for each x value
    predictions = []

    # Iterate over each value in the list of x values
    for x in x_vals:
        try:
            # Replace 'x' in the expression with the current x value
            # and evaluate the resulting mathematical expression
            predictions.append(eval(func_str.replace('x', str(x))))
        except:
            # If there is any error during evaluation (e.g., division by zero),
            # append a large error value (infinity) to the predictions
            predictions.append(float('inf'))

    # Return the list of predictions for all x values
    return predictions

def find_best_trees(tree, x_vals, y_vals):
    """ Calculate the Mean Squared Error (MSE) for the tree's predictions. """
    predictions = calculate_tree(tree, x_vals)
    mse = sum((y - pred)**2 for y, pred in zip(y_vals, predictions)) / len(y_vals)
    return mse.real / 10  # Normalize MSE and ensure it is a real number

def initialize_population(size, operators, operands):
    """ Initialize a random population of trees. """
    population = []
    for _ in range(size):
        op = random.choice(operators)
        node = Node(op)
        node.left = Node('x')
        node.right = Node(random.choice(operands) if op not in ['math.sin', 'math.cos'] else None)
        population.append(node)
    return population

def genetic_programming(x, y, generations, population_size, operators, operands):
    """ Main genetic programming loop. """
    population = initialize_population(population_size, operators, operands)
    best_tree = None
    best_mse = float('inf')

    for gen in range(generations):
        new_population = []
        for _ in range(population_size // 2):
            tree1, tree2 = random.sample(population, 2)
            new_population.extend([combine_trees(tree1, tree2), combine_trees(tree2, tree1)])

        for tree in new_population:
            mutation(tree, operators, operands)

        for tree in new_population:
            tree.MSE = find_best_trees(tree, x, y)

        population = sorted(new_population)[:population_size]

        if population[0].MSE < best_mse:
            best_mse = population[0].MSE
            best_tree = population[0]

        if best_mse < 0.5 and best_mse > -0.5:
            break

    return best_tree, best_mse, gen + 1

# Example usage
x = [round(-10 + 0.2 * i, 1) for i in range(100)]
y = [xi**2 for xi in x]

operators = ['+', '-', '*', '/', '**', 'math.sin', 'math.cos']
operands = list(range(1, 10)) + ['x']

best_tree, best_mse, best_gen = genetic_programming(x, y, 100, 100, operators, operands)

print("Best tree:", print_tree(best_tree))
print("Best MSE:", best_mse)
print("Best generation:", best_gen)

Best tree: (math.sin(math.sin(8))**x)
Best MSE: -1.2623967480194342e+27
Best generation: 100
